In [ ]:
!pip install -U google-genai
!pip install Pillow
!pip install pydantic
!pip install dotenv
!pip install numpy

In [ ]:
import os
from google import genai
from PIL import Image
from io import BytesIO
from IPython.display import display
from dotenv import load_dotenv
from google.genai import types
load_dotenv()

# --- IMPORTANT ---
# Paste your API key here. For better security, we recommend using environment variables.
# For example: API_KEY=os.environ.get("GEMINI_API_KEY")
API_KEY = os.getenv("API_KEY")
# -----------------

# Configure the client with your API key
client = genai.Client(api_key=API_KEY)

NANO_BANANA = "gemini-2.5-flash-image-preview"

In [ ]:
from IPython.display import display, Markdown, Image as IPImage
import pathlib

# Loop over all parts and display them either as text or images
def display_response(response):
  for part in response.parts:
    if part.text:
      display(Markdown(part.text))
    elif image:= part.as_image():
      display(image)
      # image.show() if not in a notebook

# Save the image
# If there are multiple ones, only the last one will be saved
def save_image(response, path):
  for part in response.parts:
    if image:= part.as_image():
      image.save(path)

import dataclasses
import numpy as np
import base64
import json
from typing import Tuple

@dataclasses.dataclass(frozen=True)
class RoomData:
  """A class to hold room data from bounding box detection."""
  y0: int
  x0: int
  y1: int
  x1: int
  label: str
  dimensions: str
  
def parse_json(json_output: str):
    """Parses JSON output from the model, removing markdown fencing."""
    if "```json" in json_output:
        json_output = json_output.split("```json")[1].split("```")[0]
    try:
        return json.loads(json_output)
    except json.JSONDecodeError:
        print(f"Warning: Could not parse JSON: {json_output}")
        return []
    

def parse_room_data(
    predicted_str: str, *, img_height: int, img_width: int, expand_percent: int = 0
) -> list[RoomData]:
  """Parses the model's string output to a list of RoomData objects."""
  items = parse_json(predicted_str)
  rooms = []
  for item in items:
    try:
        y0 = int(item["box_2d"][0] / 1000 * img_height)
        x0 = int(item["box_2d"][1] / 1000 * img_width)
        y1 = int(item["box_2d"][2] / 1000 * img_height)
        x1 = int(item["box_2d"][3] / 1000 * img_width)

        if y0 >= y1 or x0 >= x1:
            continue

        # Expand the bounding box
        if expand_percent > 0:
            dx = (x1 - x0) * (expand_percent / 100) / 2
            dy = (y1 - y0) * (expand_percent / 100) / 2
            x0 = max(0, int(x0 - dx))
            y0 = max(0, int(y0 - dy))
            x1 = min(img_width, int(x1 + dx))
            y1 = min(img_height, int(y1 + dy))

        label = item.get("label", "Unknown")
        dimensions = item.get("dimensions", "N/A")
        rooms.append(RoomData(y0, x0, y1, x1, label, dimensions))
    except (KeyError, IndexError) as e:
        print(f"Skipping an item due to parsing error: {e}")
        continue
  return rooms

def create_isolated_floor_plan_from_bbox(original_plan: Image.Image, bbox: tuple) -> Image.Image:
    """Creates an isolated floor plan by cropping to the bounding box."""
    # bbox is (x0, y0, x1, y1)
    return original_plan.crop(bbox)



def overlay_bounding_boxes_on_image(image: Image.Image, rooms: list) -> Image.Image:
    """Overlays bounding boxes on an image for visualization."""
    from PIL import ImageDraw, ImageColor
    
    overlay_image = image.copy().convert("RGB")
    draw = ImageDraw.Draw(overlay_image)
    
    colors = ['red', 'green', 'blue', 'yellow', 'purple', 'orange']
    
    for i, room in enumerate(rooms):
        color = colors[i % len(colors)]
        
        # Draw bounding box
        draw.rectangle(
            ((room.x0, room.y0), (room.x1, room.y1)),
            outline=color,
            width=3
        )
        try:
            # Attempt to load a font, fallback to default if not available
            from PIL import ImageFont
            font = ImageFont.load_default(size=16)
        except Exception:
            font = None
        draw.text((room.x0 + 5, room.y0 + 5), room.label, fill=color, font=font)
        
    return overlay_image




## Pipeline

In [ ]:
from pydantic import BaseModel, Field
import uuid
import time
import pathlib

# --- Pipeline Inputs ---
FLOOR_PLAN_IMAGE = "../plans/plan5.png"
STYLE = "2020 modern and minimalist"
# ---------------------

# Create a directory to store the output images
output_dir = "property_tour"
pathlib.Path(output_dir).mkdir(exist_ok=True)

# Display the input floor plan
print("Input Floor Plan:")
display(Image.open(FLOOR_PLAN_IMAGE))


In [ ]:
# --- 1. Detect rooms using bounding boxes ---

print("Step 1: Detecting rooms on floor plan...")

class RoomSegment(BaseModel):
    """Represents a single detected room from the floor plan."""
    label: str = Field(description="A descriptive name for the room (e.g., 'Living Room', 'Bedroom 1').")
    box_2d: list[int] = Field(description="The 2D bounding box coordinates [y0, x0, y1, x1].")
    dimensions: str = Field(description="The inferred dimensions of the room as a string (e.g., '13ft 4in x 9ft 0in').")

segmentation_prompt = """
Analyze the provided floor plan. Identify every enclosed area.
For each area, provide a bounding box and infer its dimensions from any text labels present.
If a space contains multiple functions without walls (e.g., kitchen and dining), label it as a single combined space like "Kitchen/Dining Area". 
The walls are the determining factor for separate rooms. Do combine the kitchen and living room if no wall separates them.
Include walls, doors and windows in the bounding box.
"""

original_plan_image = Image.open(FLOOR_PLAN_IMAGE).convert("RGB")
w, h = original_plan_image.size

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[segmentation_prompt, original_plan_image],
    config={
        "response_mime_type": "application/json",
        "response_schema": list[RoomSegment],
        "temperature": 0.4,
        "thinking_config": {"thinking_budget": -1}
    },
)

room_detections = parse_room_data(response.text, img_height=h, img_width=w, expand_percent=5)

# --- Display bounding boxes for debugging ---
print("\nDetection results with bounding boxes overlaid:")
debug_image = overlay_bounding_boxes_on_image(original_plan_image, room_detections)
display(debug_image)


# --- 2. Process detection results and create room list ---
print("\nStep 2: Processing detections and creating isolated floor plans...")

class Room:
    def __init__(self, name, bbox, dimensions):
        self.room_id = str(uuid.uuid4())[:8]
        self.room_name = name
        self.bbox = bbox # (x0, y0, x1, y1)
        self.dimensions = dimensions
        self.isolated_plan_path = f"{output_dir}/{self.room_id}_plan.png"
        self.unfurnished_iso_path = f"{output_dir}/{self.room_id}_unfurnished_iso.png"
        self.furnished_iso_path = f"{output_dir}/{self.room_id}_furnished_iso.png"

room_list = [Room(name=rd.label, bbox=(rd.x0, rd.y0, rd.x1, rd.y1), dimensions=rd.dimensions) for rd in room_detections]

for room in room_list:
    print(f"- Found Room: {room.room_name} (ID: {room.room_id}, Dimensions: {room.dimensions})")
    isolated_image = create_isolated_floor_plan_from_bbox(original_plan_image, room.bbox)
    isolated_image.save(room.isolated_plan_path)
    print(f"  -> Saved isolated plan to {room.isolated_plan_path}")
    display(isolated_image)

time.sleep(3)

# --- 3. Generate textual description of the decoration style ---
print(f"\nStep 3: Generating style description for '{STYLE}'...")
style_prompt = f"""
Generate a detailed, concise description for a '{STYLE}' interior design style. Focus on:
- Color palette
- Furniture style (materials, shapes)
- Lighting and accessories
This will guide image generation. Do not add any intro or outro.
"""
response = client.models.generate_content(model="gemini-2.5-flash", contents=style_prompt)
style_description = response.text
print("Style Description Generated:")
display(Markdown(style_description))


# --- 4. Generate views for each room and assemble the final property ---
print("\nStep 4: Generating individual room views...")

for room in room_list:
    print(f"\n--- Processing Room: {room.room_name} ({room.room_id}) ---")
    
    # 4a. Generate unfurnished isometric view from the isolated plan
    print(f"  4a. Generating unfurnished 3D view for {room.room_name}...")
    unfurnished_iso_prompt = f"""
    Generate a clean, unfurnished 3D isometric view of the room shown in this cropped floor plan.
    The room is the '{room.room_name}' and its dimensions are approximately {room.dimensions}. Only model the room itself. Walls are the boundaries.
    - Show only the walls and floor based on the visible plan. Do not invent any new walls or structures.
    - Do not include any furniture, decorations, or ceiling. Only include equipment if clearly shown in the plan.
    - The background must be plain white.
    - Do not add any text or labels to the image.
    - Pay close attention to the placement of doors and windows from the plan.
    """
    response = client.models.generate_content(
        model=NANO_BANANA,
        contents=[unfurnished_iso_prompt, original_plan_image, Image.open(room.isolated_plan_path)]
    )
    save_image(response, room.unfurnished_iso_path)
    print(f"  -> Saved to {room.unfurnished_iso_path}")
    display(Image.open(room.unfurnished_iso_path))
    time.sleep(5)

    # 4b. Decorate and furnish the room
    print(f"  4b. Furnishing {room.room_name} in '{STYLE}' style...")
    furnish_prompt = f"""
    Take this unfurnished 3D isometric view of the '{room.room_name}' and furnish it completely according to the style description below.
    The final image must be a photorealistic, beautifully decorated room. Maintain perfect consistency with the room's structure (walls, windows).

    Style Description:
    {style_description}
    """
    response = client.models.generate_content(
        model=NANO_BANANA,
        contents=[furnish_prompt, Image.open(room.unfurnished_iso_path)]
    )
    save_image(response, room.furnished_iso_path)
    print(f"  -> Saved to {room.furnished_iso_path}")
    display(Image.open(room.furnished_iso_path))
    time.sleep(5)

    # 4c. Generate interior eye-level views
    print(f"  4c. Generating interior shots for {room.room_name}...")
    interior_shot_prompt = f"""
    Based on this furnished isometric view of the '{room.room_name}', generate photorealistic, human-eye-level images from inside the room, each from a different angle.
    These should look like professional real estate photos. Maintain extreme consistency in style, furniture, and colors with the provided isometric view.
    Place yourself as a human in the room, looking around. RESPECTING THIS VIEW ANGLE AND LAYOUT IS CRUCIAL.
    """
    response = client.models.generate_content(
        model=NANO_BANANA,
        contents=[interior_shot_prompt, Image.open(room.furnished_iso_path)]
    )
    for i, part in enumerate(response.parts):
        if image := part.as_image():
            interior_shot_path = f"{output_dir}/{room.room_id}_interior_{i+1}.png"
            image.save(interior_shot_path)
            print(f"  -> Saved interior shot {i+1} to {interior_shot_path}")
            display(Image.open(interior_shot_path))
    time.sleep(5)

# --- 5. Generate the final assembled 3D isometric view ---
print("\nStep 5: Generating final assembled 3D view of the full property...")

assembly_prompt_parts = [
    f"""
    Assemble a single, complete 3D isometric view of the entire property.
    Use the original floor plan for the overall layout and positioning.
    Use the following furnished isometric room views to fill in the details for each corresponding room.
    The final image must be a cohesive, photorealistic, and beautifully decorated view of the entire floor, with all rooms furnished as shown in their individual images.
    Ensure all rooms are correctly placed and oriented relative to each other, as per the original floor plan.
    """,
    Image.open(FLOOR_PLAN_IMAGE)
]

# Add all the furnished room images to the prompt
for room in room_list:
    assembly_prompt_parts.append(Image.open(room.furnished_iso_path))

response = client.models.generate_content(
    model=NANO_BANANA,
    contents=assembly_prompt_parts
)

final_isometric_view_path = f"{output_dir}/final_full_furnished_view.png"
save_image(response, final_isometric_view_path)

print("Final assembled view generated and saved.")
display(Image.open(final_isometric_view_path))

print("\n--- Pipeline execution complete! ---")
print(f"All generated images are saved in the '{output_dir}' directory.")